In [ ]:
import sys
from pathlib import Path
import pickle
import pandas as pd
import geopandas as gpd

import src.bikewaysim_functions as bikewaysim_functions

import sys
sys.path.insert(0,str(Path.cwd().parent))
import file_structure_setup
config = file_structure_setup.filepaths()

import impedance_calibration.stochastic_optimization as stochastic_optimization
from network.src import modeling_turns


In [ ]:
# import matched ODS (i.e. already have the correct network nodes)
od_matrix = pd.read_csv(config['bikewaysim_fp']/'od_matrix.csv')
ods = list(set(zip(od_matrix['orig_N'],od_matrix['dest_N'])))


In [ ]:
# import test case/ base case (think of a better word) network
with (config['calibration_fp']/"calibration_network.pkl").open('rb') as fh:
    links,turns = pickle.load(fh)
# import the calibration results
with (config['calibration_fp']/"calibration_results.pkl").open('rb') as fh:
    calibration_results = pickle.load(fh)
turn_G = modeling_turns.make_turn_graph(turns)

In [ ]:
#remove imporvements that aren't improvements
links.loc[links['facility_fwd']==links['improvement'],'improvement'] = None
links.loc[links['improvement'].notna(),['facility_fwd','improvement']]

In [ ]:
#change cycletracks to multi use paths for now
change_infra = {
    'buffered bike lane': 'bike lane',
    'cycletrack': 'multi use path'
}
links['improvement'] = links['improvement'].apply(lambda x: change_infra.get(x,x))
links['facility_fwd'] = links['facility_fwd'].apply(lambda x: change_infra.get(x,x))

In [ ]:
#combine facilities into one
import numpy as np
links['facility_fwd_new'] = np.where(links['improvement'].notna(),links['improvement'],links['facility_fwd'])
# create the neccessary attributes
links['bike lane'] = (links['facility_fwd'] == 'bike lane').astype(int)
links['multi use path'] = (links['facility_fwd'] == 'multi use path').astype(int)
links.loc[links['multi use path']==1,'lanes'] = 0

In [ ]:
#choose the calibration coefficients to use
calibration_results


In [ ]:
calibration_result = calibration_results[-1]
calibration_result

In [ ]:
betas = list(calibration_result.values())[0:-3]
betas_links = calibration_result['beta_links']
betas_turns = calibration_result['beta_turns']
betas


# Shortest Path Routing


In [ ]:
from importlib import reload
reload(stochastic_optimization)

# run impedance routing on all the provided ODs
base_impedance_col = "travel_time_min"
stochastic_optimization.back_to_base_impedance(base_impedance_col,links,turns,turn_G)

#update impedances
print(betas)
stochastic_optimization.impedance_update(betas,betas_links,betas_turns,
                          stochastic_optimization.link_impedance_function,
                          base_impedance_col,
                          stochastic_optimization.turn_impedance_function,
                          links,turns,turn_G)
current_links = links.copy()


In [ ]:
results_dict = {(start_node,end_node):stochastic_optimization.impedance_path(turns,turn_G,links,start_node,end_node) for start_node, end_node in ods}


In [ ]:
with (config['bikewaysim_fp']/"current_results.pkl").open('wb') as fh:
    pickle.dump(results_dict,fh)


# Repeat with Network Improvements


In [ ]:
#update the bike lane, multi use path, and lanes attributes
links['bike lane'] = (links['facility_fwd_new']=='bike lane').astype(int)
links['multi use path'] = (links['facility_fwd_new']=='multi use path').astype(int)

#for multiuse paths, make sure to set the lanes attribute to zero
links.loc[links['multi use path']==1,'lanes'] = 0


In [ ]:
from importlib import reload
reload(stochastic_optimization)

# run impedance routing on all the provided ODs
base_impedance_col = "travel_time_min"
stochastic_optimization.back_to_base_impedance(base_impedance_col,links,turns,turn_G)

#update impedances
#betas = #past_betas[np.array(past_vals).argmin()]#x.x
print(betas)
stochastic_optimization.impedance_update(betas,betas_links,betas_turns,
                          stochastic_optimization.link_impedance_function,
                          base_impedance_col,
                          stochastic_optimization.turn_impedance_function,
                          links,turns,turn_G)
future_links = links.copy()


In [ ]:
results_dict = {(start_node,end_node):stochastic_optimization.impedance_path(turns,turn_G,links,start_node,end_node) for start_node, end_node in ods}


In [ ]:

with (config['bikewaysim_fp']/"future_results.pkl").open('wb') as fh:
    pickle.dump(results_dict,fh)